<a href="https://colab.research.google.com/github/sravanthivalluru06/OIB-SIP/blob/main/Project_4_Proposal_Level_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install pandas numpy plotly matplotlib nltk kaggle

KAGGLE API SETUP

DATASET PATH SETUP

In [ ]:
APPS_PATH = "/content/data/datasets/apps.csv"
REVIEWS_PATH = "/content/data/datasets/user_reviews.csv"

import os
if not os.path.exists(APPS_PATH):
    raise FileNotFoundError(f"APPS_PATH not found: {APPS_PATH}\nCheck the printed files in /data and update APPS_PATH.")
if REVIEWS_PATH is not None and not os.path.exists(REVIEWS_PATH):
    print(f"[warn] REVIEWS_PATH not found: {REVIEWS_PATH} — proceeding without reviews.")
    REVIEWS_PATH = None

OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)


In [ ]:
import re, warnings
from typing import Optional
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

def _standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(r"[^a-z0-9]+", "_", regex=True)
        .str.strip("_")
    )
    return df

def _coerce_numeric(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series, errors="coerce")

def _parse_size_to_mb(x: Optional[str]) -> float:
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    if "varies" in s or s in {"nan", ""}:
        return np.nan
    m = re.match(r"([\d\.]+)\s*([mk]|mb|kb)?", s)
    if not m:
        return np.nan
    val = float(m.group(1))
    unit = m.group(2) or ""
    if unit in {"m", "mb"}:
        return val
    if unit in {"k", "kb"}:
        return val / 1024.0
    return val

def _parse_installs(x: Optional[str]) -> float:
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower().replace("+", "").replace(",", "").replace(" ", "")
    if re.match(r"^\d+$", s):
        return float(s)
    return pd.to_numeric(s, errors="coerce")

def _parse_price(x: Optional[str]) -> float:
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s.lower() in {"free", "0", "0.0"}:
        return 0.0
    s = re.sub(r"[^0-9\.\-]", "", s)
    return pd.to_numeric(s, errors="coerce")


def clean_apps_df(df_apps_raw: pd.DataFrame) -> pd.DataFrame:
    df = _standardize_columns(df_apps_raw)

    col_map = {
        "app": ["app", "app_name", "name", "title"],
        "category": ["category", "categories"],
        "rating": ["rating", "ratings"],
        "reviews": ["reviews", "review_count", "total_reviews"],
        "size": ["size", "app_size"],
        "installs": ["installs", "install", "num_installs"],
        "type": ["type"],
        "price": ["price"],
        "content_rating": ["content_rating"],
        "genres": ["genres", "genre"],
        "last_updated": ["last_updated", "updated"],
        "android_ver": ["android_ver", "minimum_android", "android_version"],
    }
    for new_name, candidates in col_map.items():
        found = None
        for c in candidates:
            if c in df.columns:
                found = c
                break
        if found and found != new_name:
            df = df.rename(columns={found: new_name})

    if "rating" in df.columns:
        df["rating"] = _coerce_numeric(df["rating"]).clip(lower=0, upper=5)
    if "reviews" in df.columns:
        df["reviews"] = _coerce_numeric(df["reviews"])
    if "size" in df.columns:
        df["size_mb"] = df["size"].apply(_parse_size_to_mb)
    elif "size_mb" not in df.columns:
        df["size_mb"] = np.nan
    if "installs" in df.columns:
        df["installs"] = df["installs"].apply(_parse_installs)
    if "price" in df.columns:
        df["price"] = df["price"].apply(_parse_price)
    else:
        df["price"] = np.nan

    if "type" in df.columns:
        df["type"] = df["type"].astype(str).str.strip().str.title()
    else:
        df["type"] = np.where((df["price"].fillna(0) > 0), "Paid", "Free")

    subset = [c for c in ["app", "category"] if c in df.columns]
    if subset:
        df = df.drop_duplicates(subset=subset, keep="first")

    if "rating" in df.columns:
        if "category" in df.columns:
            df["rating"] = df.groupby("category")["rating"].transform(lambda s: s.fillna(s.median()))
        df["rating"] = df["rating"].fillna(df["rating"].median())

    df["is_free"] = (df["price"].fillna(0) == 0).astype(int)
    df["price_bucket"] = pd.cut(
        df["price"].fillna(0),
        bins=[-0.01, 0, 0.99, 4.99, 9.99, 19.99, 49.99, np.inf],
        labels=["Free", "≤$0.99", "$1–4.99", "$5–9.99", "$10–19.99", "$20–49.99", "$50+"],
    )
    return df


def summarize_categories(df: pd.DataFrame) -> pd.DataFrame:
    agg = {
        "app": "count",
        "rating": "mean",
        "installs": "median",
        "is_free": "mean",
        "price": "median",
        "size_mb": "median",
        "reviews": "median",
    }
    present_cols = {k: v for k, v in agg.items() if k in df.columns}
    s = (
        df.groupby(["category"])
          .agg(present_cols)
          .rename(columns={
              "app": "num_apps",
              "rating": "avg_rating",
              "installs": "median_installs",
              "is_free": "share_free",
              "price": "median_price",
              "size_mb": "median_size_mb",
              "reviews": "median_reviews",
          })
          .reset_index()
          .sort_values("num_apps", ascending=False)
    )
    return s

def summarize_by_price_bucket(df: pd.DataFrame) -> pd.DataFrame:
    if "price_bucket" not in df.columns:
        return pd.DataFrame()
    return (
        df.groupby(["category", "price_bucket"])
          .agg(num_apps=("app", "count"),
               avg_rating=("rating", "mean"),
               median_installs=("installs", "median"))
          .reset_index()
    )


def ensure_vader_downloaded():
    try:
        nltk.data.find("sentiment/vader_lexicon.zip")
    except LookupError:
        nltk.download("vader_lexicon")

def load_and_prepare_reviews(path: str) -> Optional[pd.DataFrame]:
    if not path or not os.path.exists(path):
        return None
    reviews = pd.read_csv(path, encoding="utf-8", low_memory=False)
    reviews = _standardize_columns(reviews)

    text_col = None
    for c in ["translated_review", "review", "content"]:
        if c in reviews.columns:
            text_col = c
            break
    if text_col is None:
        return None

    reviews = reviews.dropna(subset=[text_col])

    if "sentiment" not in reviews.columns or reviews["sentiment"].isna().all():
        ensure_vader_downloaded()
        sia = SentimentIntensityAnalyzer()
        reviews["compound"] = reviews[text_col].astype(str).apply(lambda s: sia.polarity_scores(s)["compound"])
        reviews["sentiment"] = pd.cut(
            reviews["compound"],
            bins=[-1.0, -0.05, 0.05, 1.0],
            labels=["negative", "neutral", "positive"],
            include_lowest=True
        ).astype(str)
    else:
        reviews["sentiment"] = reviews["sentiment"].astype(str).str.lower().str.strip()

    keep = ["app", text_col, "sentiment"]
    keep = [c for c in keep if c in reviews.columns]
    return reviews[keep]

def sentiment_summary(reviews: pd.DataFrame) -> pd.DataFrame:
    s = (reviews.groupby(["app", "sentiment"]).size().unstack(fill_value=0).reset_index())
    for col in ["negative", "neutral", "positive"]:
        if col not in s.columns:
            s[col] = 0
    s["total_reviews"] = s[["negative", "neutral", "positive"]].sum(axis=1)
    s["share_positive"] = s["positive"] / s["total_reviews"].replace(0, np.nan)
    return s


def _save_plotly(fig, path_html: str, title: str = ""):
    if title:
        fig.update_layout(title=title)
    fig.write_html(path_html, include_plotlyjs="cdn")

def visualize_category_distribution(df: pd.DataFrame, out_dir: str):
    if "category" not in df.columns:
        return None
    top_counts = df["category"].value_counts().reset_index()
    top_counts.columns = ["category", "num_apps"]
    fig = px.bar(
        top_counts.sort_values("num_apps", ascending=False),
        x="category", y="num_apps",
        hover_data=["num_apps"],
        labels={"category": "Category", "num_apps": "Number of Apps"},
    )
    fig.update_layout(xaxis_tickangle=45, title="App Distribution by Category")
    _save_plotly(fig, os.path.join(out_dir, "chart_category_distribution.html"))
    return fig

def visualize_rating_vs_installs(df: pd.DataFrame, out_dir: str):
    if not {"rating", "installs"}.issubset(df.columns):
        return None
    fig = px.scatter(
        df, x="installs", y="rating",
        hover_data=[c for c in ["app", "category", "price"] if c in df.columns],
        labels={"installs": "Installs (log scale)", "rating": "Rating (0–5)"},
    )
    fig.update_xaxes(type="log")
    fig.update_layout(title="Ratings vs. Installs (log scale)")
    _save_plotly(fig, os.path.join(out_dir, "chart_rating_vs_installs.html"))
    return fig

def visualize_price_distribution_by_category(df: pd.DataFrame, out_dir: str):
    if not {"category", "price_bucket"}.issubset(df.columns):
        return None
    grouped = df.groupby(["category", "price_bucket"]).size().reset_index(name="num_apps")
    fig = px.treemap(
        grouped, path=["category", "price_bucket"], values="num_apps",
        hover_data=["num_apps"],
        labels={"num_apps": "Number of Apps"}
    )
    fig.update_layout(title="Price Distribution within Categories")
    _save_plotly(fig, os.path.join(out_dir, "chart_price_distribution_by_category.html"))
    return fig

def visualize_size_vs_rating(df: pd.DataFrame, out_dir: str):
    if not {"size_mb", "rating"}.issubset(df.columns):
        return None
    fig = px.scatter(
        df.dropna(subset=["size_mb", "rating"]),
        x="size_mb", y="rating",
        hover_data=[c for c in ["app", "category", "price"] if c in df.columns],
        labels={"size_mb": "App Size (MB)", "rating": "Rating (0–5)"},
    )
    fig.update_layout(title="App Size vs. Ratings")
    _save_plotly(fig, os.path.join(out_dir, "chart_size_vs_rating.html"))
    return fig

def visualize_sentiment_by_category(df_apps: pd.DataFrame, reviews_summary: pd.DataFrame, out_dir: str):
    if "app" not in df_apps.columns or reviews_summary is None or reviews_summary.empty:
        return None
    merged = pd.merge(
        df_apps[["app", "category"]].drop_duplicates(),
        reviews_summary[["app", "share_positive"]],
        on="app", how="left"
    ).dropna(subset=["share_positive"])
    if merged.empty:
        return None
    cat_sent = merged.groupby("category")["share_positive"].mean().reset_index()
    fig = px.bar(
        cat_sent.sort_values("share_positive", ascending=False),
        x="category", y="share_positive",
        hover_data=["share_positive"],
        labels={"category": "Category", "share_positive": "Avg. Share Positive Reviews"},
    )
    fig.update_layout(xaxis_tickangle=45, title="Average Share of Positive Reviews by Category")
    _save_plotly(fig, os.path.join(out_dir, "chart_sentiment_by_category.html"))
    return fig


In [ ]:
df_apps_raw = pd.read_csv(APPS_PATH, encoding="utf-8", low_memory=False)
print("[info] raw apps shape:", df_apps_raw.shape)

[info] raw apps shape: (9659, 14)


In [ ]:
df_apps = clean_apps_df(df_apps_raw)
print("[info] cleaned apps shape:", df_apps.shape)
df_apps.to_csv(os.path.join(OUT_DIR, "cleaned_apps.csv"), index=False)


[info] cleaned apps shape: (9659, 17)


In [ ]:
cat_summary = summarize_categories(df_apps)
cat_summary.to_csv(os.path.join(OUT_DIR, "category_summary.csv"), index=False)
display(cat_summary.head(5))

price_bucket_summary = summarize_by_price_bucket(df_apps)
if not price_bucket_summary.empty:
    price_bucket_summary.to_csv(os.path.join(OUT_DIR, "price_buckets_summary.csv"), index=False)
    display(price_bucket_summary.head(5))

fig1 = visualize_category_distribution(df_apps, OUT_DIR)
fig2 = visualize_rating_vs_installs(df_apps, OUT_DIR)
fig3 = visualize_price_distribution_by_category(df_apps, OUT_DIR)
fig4 = visualize_size_vs_rating(df_apps, OUT_DIR)

for fig in [fig1, fig2, fig3, fig4]:
    if fig is not None:
        fig.show()

,category,num_apps,avg_rating,median_installs,share_free,median_price,median_size_mb,median_reviews
11,FAMILY,1832,4.194378,50000.0,0.900109,0.0,19.0,649.0
14,GAME,959,4.249948,1000000.0,0.914494,0.0,38.0,28510.0
29,TOOLS,827,4.060701,50000.0,0.905683,0.0,4.2,478.0
4,BUSINESS,420,4.136429,1000.0,0.971429,0.0,8.6,15.0
20,MEDICAL,395,4.202025,1000.0,0.789873,0.0,15.0,25.0


/tmp/ipython-input-2043958430.py:155: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["category", "price_bucket"])


,category,price_bucket,num_apps,avg_rating,median_installs
0,ART_AND_DESIGN,Free,61,4.340984,100000.0
1,ART_AND_DESIGN,≤$0.99,0,NaN,NaN
2,ART_AND_DESIGN,$1–4.99,3,4.733333,5000.0
3,ART_AND_DESIGN,$5–9.99,0,NaN,NaN
4,ART_AND_DESIGN,$10–19.99,0,NaN,NaN


/tmp/ipython-input-2043958430.py:248: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/usr/local/lib/python3.12/dist-packages/plotly/express/_core.py:1727: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



EXPORT RESULTS

In [ ]:
import shutil
from google.colab import files

archive_path = shutil.make_archive("google_play_outputs", "zip", base_dir=OUT_DIR)
files.download(archive_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>